# 0. Name & Z-ID
Matt Borek

z1951125

# Project Gutenberg Dataset

In [401]:
import os
from urllib.request import urlretrieve

def download_text(url):
    local_fname = os.path.basename(url)
    if not os.path.exists(local_fname):
        urlretrieve(url, local_fname)
    return local_fname

In [402]:
fname = download_text('https://www.gutenberg.org/files/17610/17610-8.txt') #(encoding: iso-8859-1)
fname2 = download_text('https://www.gutenberg.org/files/63438/63438-0.txt') #(encoding: utf-8-sig)
fname3 = download_text('https://www.gutenberg.org/files/63439/63439-0.txt') #(encoding: utf-8-sig)

# 1. Read Data

In [403]:
import re

def read_data(filename, encoding):
    with open(filename, 'r', encoding=encoding) as text:
        section_found = False
        text_lines = []
        for line in text:
            if re.match(r'\*{3} START OF THIS PROJECT GUTENBERG EBOOK .*? \*{3}', line):
                section_found = True
                continue
            
            if re.match(r'\*{3} END OF THIS PROJECT GUTENBERG EBOOK .*? \*{3}', line):
                section_found = False
                break
            
            if section_found:
                text_lines.append(line.strip())
    return text_lines

In [404]:
#for line in read_data(fname,'iso-8859-1'): #book1
#    print(line)
    
#for line in read_data(fname2,'utf-8-sig'): #book2
#    print(line)
    
#for line in read_data(fname2,'utf-8-sig'): #book3
#    print(line)

fname = read_data(fname,'iso-8859-1')
fname2 = read_data(fname2,'utf-8-sig')
fname3 = read_data(fname3,'utf-8-sig')

# 2. Find Non-ASCII Characters

In [405]:
def find_non_ASCII(text_lines):
    non_ASCII = set()
    for line in text_lines:
        for char in line:
            if ord(char) > 128:
                non_ASCII.add(char)
    return non_ASCII

In [406]:
print(len(find_non_ASCII(fname)))
print(len(find_non_ASCII(fname2)))
print(len(find_non_ASCII(fname3)))

22
46
25


# 3. Convert Quotes

In [407]:
def convert_quotes(text_lines):
    conversions = {'‘':'\'',
                   '’':'\'',
                   '“':'"',
                   '”':'"',
                   '«':'"',
                   '»':'"',
                }
    return [''.join([(char if char not in conversions else conversions[char]) for char in line]) for line in text_lines]

In [408]:
fname = convert_quotes(fname)
fname2 = convert_quotes(fname2)
fname3 = convert_quotes(fname3)